## Este código faz parte do teste de aptidão técnica proposto pela empresa Docket.
Para realização do teste foi implementado uma técnica bastante utilizada por meio de processamento de linguagem natural (PLN), chamada de análise de sentimentos, para interpretar notícias do mercado de ações e classificá-las como positiva, negativa e neutra. Para isso, nós utilizamos a biblioteca Keras e comparamos o desempenho de dois modelos (Rede neural recorrente com camada única LSTM e camada Bidirecional).

Importação das bibliotecas

In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf

# import contractions

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

2022-12-03 21:41:34.348511: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/wanderson/openssl/lib:/home/wanderson/openssl/lib/
2022-12-03 21:41:34.348552: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1131)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1131)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     una

False

### Carregamento do dataset via gdrive

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

A importação do conjunto de dados é feita no modo hard-code. Em um projeto mais estruturado, é recomendável que essas informações sejam armazenadas em uma variável de ambiente da própria máquina ou armazena-lás nas variáveis de ambiente do GitHub, na seção de Secrets -> Actions.

In [3]:
df = pd.read_csv('../data/processed/financial_phrase_bank_pt_br.csv')
df

,y,text,text_pt
0,neutral,Technopolis plans to develop in stages an area...,A Technopolis planeja desenvolver em etapas um...
1,negative,The international electronic industry company ...,"A Elcoteq, empresa internacional da indústria ..."
2,positive,With the new production plant the company woul...,Com a nova planta de produção a empresa aument...
3,positive,According to the company 's updated strategy f...,De acordo com a estratégia atualizada da empre...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,FINANCIAMENTO DO CRESCIMENTO DA ASPOCOMP A Asp...
...,...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...,LONDRES MarketWatch - Os preços das ações term...
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,"As vendas de cerveja da Rinkuskiai caíram 6,5 ..."
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"O lucro operacional caiu para EUR 35,4 milhões..."
4843,negative,Net sales of the Paper segment decreased to EU...,As vendas líquidas do segmento de Papel diminu...


Distribuição das labels (value.counts())

Esse método permite contabilizar a quantidade de mensagens positivas, negativas e neutras.

In [4]:
print(df['y'].value_counts())

neutral     2878
positive    1363
negative     604
Name: y, dtype: int64


## **PRÉ PROCESSAMENTO**

A etapa de pré processamento dos dados consiste em um conjunto de atividades que envolvem conversão de dados brutos em dados prontos para uso, ou seja, é um processo que compreende a preparação, organização e estrutura de dados.
As etapas do pré processamento utilizados foram:
- **Limpeza textual**: Nesta etapa, serão removidos os caracteres especiais, números e pontuações;
- **Caixa baixa**: Uniformizar o texto com letras minúsculas;
- **Tokenização**: O ato de separar o texto em unidades menores chamadas tokens.Esses tokens pode ser palavras ou caracteres;
- **Remoção de Stopwords**: Remover palavras que não acresentam ao projeto;
- **Lematização**: É uma técnica de normalização usada para flexionar a palavra de forma que possam ser analisadas como um único item. Há uma outra técnica bastante utilizada chamada de Stemming, eu optei por não utilizar o Stemming devido a perda de informação ocasionada pela redicalização das palavras.
- Considerar palavras maiores que 1 caractere.


In [5]:
def clean_text(text):
    """
    Take string input and return a clean text without punctuation,
    remove numbers, transforms into lower case and remove extra whitespace.
    """
    clean = re.sub("[^a-zA-z\s]", "", text.lower())
    blank_space = re.sub("\s+", " ", clean)
    return blank_space


def remove_stopwords(sentence):
    """
    removes all the stop words like "is,the,a, etc."
    """
    stop_words = stopwords.words("english")
    return [w for w in nltk.word_tokenize(sentence) if not w in stop_words]


def lemmatize(text):
    """
    Grouping the inflected parts of a word so that they can be analyzed
    as a single item
    """
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemmatized_word


def greater_than_1(text):
    """
    Removes words that contain only 1 character
    """
    return [w for w in text if len(w) > 1]


def word_tokenize(text):
    """
    :param text:
    :return: list of words
    """
    return nltk.word_tokenize(text)


### Realiza o pré processamento do texto e acrescenta mais uma coluna chamada de "proprocessed"

In [6]:
def preprocess(text):
    clean_text_ = clean_text(text)
    remove_stop = remove_stopwords(clean_text_)
    lemma = lemmatize(remove_stop)
    less = greater_than_1(lemma)
    join_tokens = " ".join(less)

    return join_tokens


df["preprocessed"] = df["text"].apply(preprocess)
df

,y,text,text_pt,preprocessed
0,neutral,Technopolis plans to develop in stages an area...,A Technopolis planeja desenvolver em etapas um...,technopolis plan develop stage area le square ...
1,negative,The international electronic industry company ...,"A Elcoteq, empresa internacional da indústria ...",international electronic industry company elco...
2,positive,With the new production plant the company woul...,Com a nova planta de produção a empresa aument...,new production plant company would increase ca...
3,positive,According to the company 's updated strategy f...,De acordo com a estratégia atualizada da empre...,according company updated strategy year baswar...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,FINANCIAMENTO DO CRESCIMENTO DA ASPOCOMP A Asp...,financing aspocomp growth aspocomp aggressivel...
...,...,...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...,LONDRES MarketWatch - Os preços das ações term...,london marketwatch share price ended lower lon...
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,"As vendas de cerveja da Rinkuskiai caíram 6,5 ...",rinkuskiai beer sale fell per cent million lit...
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"O lucro operacional caiu para EUR 35,4 milhões...",operating profit fell eur mn eur mn including ...
4843,negative,Net sales of the Paper segment decreased to EU...,As vendas líquidas do segmento de Papel diminu...,net sale paper segment decreased eur mn second...


In [26]:
df['text'][4807]

'Bosse added that Trygvesta does not have the financial strength to acquire the entire unit .'

## Label encoding
Como a estrutura do conjunto de dados é baseada em 3 categorias, foi necessário converter as labels (neutral, negative e positive) para o tipo float de forma que o modelo possa interpretar. Para realizar essa tarefa, foi usado o método to_categorical do Keras

In [7]:
labels = np.array(df['y'])
y = []
for i in range(len(labels)):
    if labels[i] == 'neutral':
        y.append(0)
    if labels[i] == 'negative':
        y.append(1)
    if labels[i] == 'positive':
        y.append(2)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")
del y
labels.shape

(4845, 3)

## Word embeddings

É método utilizado para transformar o texto em uma informação numérica, mais especificamente um vetor. No nosso caso, o texto será transformado em tensores.

Métodos
- **fit_on_texts**: Este método cria um índice de vocabulário com base no índice de palavras
- **texts_to_sequences**: Transforma cada palavra do texto em uma sequência de inteiros. Basicamente, ele pega cada palavra no texto e a substitui pelo valor inteiro correspondente ao word_index do dicionário.

Função:
- **pad_sequences**: Essa função transforma uma lista (comprimento *num_samples*) de sequencias (lista de inteiros) em um array Numpy2D. No nosso caso, teremos um array bidimensional com o comprimento de palavras 4845, sendo o máximo 5000 e maxlen de 200. Se esse último não for fornecido, as sequências serão preenchidas até o comprimento variável da sequência individual mais longa.


In [8]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import pickle

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df["preprocessed"])
# pickle.dump(tokenizer, open('../models/tokenizer.sav', 'wb'))
sequences = tokenizer.texts_to_sequences(df["preprocessed"])
financial_text = pad_sequences(sequences, maxlen=max_len)
print(financial_text.shape)


(4845, 200)


### **Divisão dos de treinamento e teste**

É muito comum encontrar na literatura as proporções de 70% de treinamento e 30% de testes, ou 80% de treinamento e 20%, mesmo 90/10. Na verdade, esta proporção não é unânime e sua definição se adapta à realidade de cada projeto.

Em um conjunto com poucos dados de treinamento, recomenda-se priorizar a maior porcentagem de treinamento, ou seja, 80/20. Utilizamos esta distribuição no projeto atual.

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    financial_text, labels, random_state=42, test_size=0.2
)
print(len(X_train), len(X_test), len(y_train), len(y_test))

## **Arquitetura da Rede**

### **LSTM**

<ins>**Camada Embedding**:</ins>

1.1 **input_dim**: É o número de palavras únicas no vocabulário.

1.2 **output_dim**: Comprimento do vetor para cada palavra.

<ins>**LSTM**</ins>

2.1 **Camada LSTM**: As redes neurais recorrentes oferece uma vantagem pois ela permite que a informação persista a cada loop devido a sua estrutura de células. Nos argumentos, a camada LSTM foi definida como 15, que significa a quantidade de camadas ocultas na rede.

2.2 **Dropout**: Durante o processo de treinamento, algumas saídas da camada são ignoradas aleatoriamente ou "descartadas", isso significa que a rede não depende de 100% dos neurônios. Essa técnica permite a coadaptação das camadas da rede com o intuito de corrigir os erros das camadas anteriores, tornando o modelo mais robusto. Na prática, é comum observar grandes redes neurais treinadas com pequenos conjunto de dados, essa combinação pode ocasionar o superajuste dos dados de treinamento (overfitting) e consequentemente reduzir o desempenho da rede. Dessa forma, o dropout foi definido em 0.5, isso significa que 50% dos nós são desligados. Normalmente, esses valores oscilam entre 0.5 e 0.8, dependendo da característica da rede, onde 1.0 não existe dropout e 0.0 significa nenhuma saída da camada.


<ins>**Camada de activação**</ins>

3.1. **Função de ativação**: A camada de ativação foi definida com 3 classes possíveis do conjunto de dados (positive, negative e neutral). Por ser uma tarefa de classificação *multiclasse*, foi escolhido a função de ativação softmax, pois desejamos obter a distribuição das probabilidades de cada classe como saída.

<ins>**Otimizador**</ins>

4.1. **Otimizador (optimizer)**: É o mecanismo que calcula constantemente o gradiente de perda e define como se move em relação a função de perda para encontrar os mínimos globais, ou seja, encontrar os melhores parâmetros da rede. O otimizador **Adam** foi utilizado, pelo fato de convergir de forma mais rápida em relação aos otimizadores concorrentes (SGD, Momentum, RMSProp)

4.2. **Função de perda**: Para a função de perda, foi utilizado o categorical_crossentropy que é normalmente usado quando se lida com tarefas de classificação multiclasse..

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [ ]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20)) 
model1.add(layers.LSTM(15, dropout=0.5))  # Camada LSTM
model1.add(layers.Dense(3, activation="softmax")) # Camada de ativação


model1.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

checkpoint1 = ModelCheckpoint(
    "../models/best_model_test1.hdf5",
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    mode="auto",
    period=1,
    save_weights_only=False,
)
history = model1.fit(
    X_train,
    y_train,
    epochs=70,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint1],
)

### **LSTM Bidirecional**

In [ ]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20, dropout=0.6)))
model2.add(layers.Dense(3, activation="softmax"))
model2.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
checkpoint2 = ModelCheckpoint(
    "/content/gdrive/MyDrive/DOCKET/best_model_test2.hdf5",
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    mode="auto",
    period=1,
    save_weights_only=False,
)
history = model2.fit(
    X_train,
    y_train,
    epochs=70,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint2],
)


### **Comparação dos modelos**

### Modelo LSTM

In [ ]:
import keras

In [ ]:
best_model_A = keras.models.load_model("../models/best_model_test1.hdf5")
# test_loss, test_acc = best_model_A.evaluate(X_test, y_test, verbose=2)
# print("Model accuracy: ", test_acc)

### Modelo LSTM Bidirecional

In [ ]:
best_model_B = keras.models.load_model("/content/gdrive/MyDrive/DOCKET/best_model2.hdf5")
# test_loss, test_acc = best_model_B.evaluate(X_test, y_test, verbose=2)
# print("Model accuracy: ", test_acc)

### **Confusion Matrix**

Neste ponto, o melhor modelo até o momento tem sido o LSTM Bidirecional. Para entender melhor o desempenho das previsões, utilizaremos a matriz de confusão. 

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

predictions = best_model_B.predict(X_test)

matrix = confusion_matrix(
    y_test.argmax(axis=1), np.around(predictions, decimals=0).argmax(axis=1)
)

conf_matrix = pd.DataFrame(
    matrix,
    index=["Neutral", "Negative", "Positive"],
    columns=["Neutral", "Negative", "Positive"],
)
# Normalizing
conf_matrix = conf_matrix.astype("float") / conf_matrix.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(15, 15))
sns.heatmap(conf_matrix, annot=True, annot_kws={"size": 15})

Na imagem acima podemos deduzir que 60% das avaliações **positivas** foram realmente classificadas como positivas. Em relação as avaliações **negativas** 54% das avaliações foram realmente classificadas como negativas. Por fim, 84% das classificações **neutras** foram realmente classificadas como neutras. Em um cenário real, seria necessário se aproximar dos 95% de acertividade.
Percebe-se que a proporção de cada label tem relação direta com os resultados obtidos, onde a categoria com o maior número de exemplos (neutro), obteve o melhor resultado (84%). Por outro lado, a categoria com o menor número de exemplos (negativo), obteve apenas (54%) de acertividade.

Podemos considerar alguns quisitos para melhoria dos resultados
- Aplicar a técnica de data augmentation com o intuito de balancear as amostras,
- Revisar os hiperparâmetros dos modelos, inclusive testar outros otimizadores, 
- Testar outros modelos de aprendizagem de máquina.

**Testando o modelo com outras entradas**

In [ ]:
sentiment = ["Negative", "Neutral", "Positive"]
sequence = tokenizer.texts_to_sequences(
    [
        """FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs ."""
    ]
)
test = pad_sequences(sequence, maxlen=max_len)
sentiment[np.around(best_model_A.predict(test), decimals=0).argmax(axis=1)[0]]

In [ ]:
sequence = tokenizer.texts_to_sequences(
    [
        """Sales in Finland decreased by 10.5 % in January , while sales outside Finland dropped by 17 % ."""
    ]
)
test = pad_sequences(sequence, maxlen=max_len)
sentiment[np.around(best_model_A.predict(test), decimals=0).argmax(axis=1)[0]]

In [ ]:
sequence = tokenizer.texts_to_sequences(
    [
        """Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said ."""
    ]
)
test = pad_sequences(sequence, maxlen=max_len)
sentiment[np.around(best_model_A.predict(test), decimals=0).argmax(axis=1)[0]]

In [36]:
df['text'][4840]

'LONDON MarketWatch -- Share prices ended lower in London Monday as a rebound in bank stocks failed to offset broader weakness for the FTSE 100 .'